<a href="https://colab.research.google.com/github/OneFineStarstuff/OneFineStardust/blob/main/Explainable_AI_(XAI).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install keras_tuner

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.applications import VGG16
from keras_tuner.tuners import RandomSearch
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img

# Load the dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize the images to values between 0 and 1
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# Reshape data to fit model input shape (28, 28, 1)
x_train_cnn = x_train.reshape((60000, 28, 28, 1))
x_test_cnn = x_test.reshape((10000, 28, 28, 1))

# Create a validation set from the training data
x_train_cnn, x_val_cnn = x_train_cnn[:-12000], x_train_cnn[-12000:]
y_train, y_val = y_train[:-12000], y_train[-12000:]

# Create a sequential model
model_cnn = models.Sequential()
model_cnn.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model_cnn.add(layers.MaxPooling2D((2, 2)))
model_cnn.add(layers.Conv2D(64, (3, 3), activation='relu'))
model_cnn.add(layers.MaxPooling2D((2, 2)))
model_cnn.add(layers.Flatten())
model_cnn.add(layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01))) # Regularization
model_cnn.add(layers.Dropout(0.5))  # Dropout regularization
model_cnn.add(layers.Dense(10, activation='softmax'))

# Compile the model
model_cnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=False
)
datagen.fit(x_train_cnn)

# Learning rate scheduler
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

# Train the model with augmented data and learning rate scheduler
model_cnn.fit(datagen.flow(x_train_cnn, y_train, batch_size=32), epochs=5, validation_data=(x_val_cnn, y_val), callbacks=[lr_scheduler])

# Evaluate the model
test_loss_cnn, test_acc_cnn = model_cnn.evaluate(x_test_cnn, y_test)
print(f'Test accuracy: {test_acc_cnn}')

# Make predictions
predictions_cnn = model_cnn.predict(x_test_cnn)

# Plotting some test images with their predicted labels
for i in range(5):
    plt.imshow(x_test_cnn[i].reshape(28, 28), cmap='gray')
    plt.title(f'Predicted label: {np.argmax(predictions_cnn[i])}')
    plt.axis('off')
    plt.show()

# SHAP for model interpretability
import shap

# Use a smaller sample of your data to save time and memory
sample_indices = np.random.choice(x_train_cnn.shape[0], 1000, replace=False)
sample_data = x_train_cnn[sample_indices]

# Initialize DeepExplainer
explainer = shap.DeepExplainer(model_cnn, sample_data)

# Explain predictions
shap_values = explainer.shap_values(x_test_cnn[:100])

# Visualize the SHAP values
shap.summary_plot(shap_values, x_test_cnn[:100])